# Set environmental variables for local development

In [ ]:
import orjson, os
with open("local.settings.json") as f:
    os.environ.update(orjson.loads(f.read())["Values"])

# Get OnSpot Queue Stats
Legacy mode

In [ ]:
from libs.openapi.clients import OnSpotAPI
OSA = OnSpotAPI(production=True)
stat = OSA.createRequest(("/status/queue", "get"))
_, data, _ = await stat.request()
data

# Register SQL data provider
Note: this may take about 15-30 seconds to complete as it waits for the SQL database to be available and scans the provider metadata.

In [ ]:
from libs.data import from_bind, register_binding

if not from_bind("salesforce"):
    register_binding(
        "salesforce",
        "Structured",
        "sql",
        url=os.environ["DATABIND_SQL_SALESFORCE"],
        schemas=["dbo"],
)

# Instantiate data provider session

In [ ]:
from libs.data.structured.sqlalchemy import SQLAlchemyStructuredProvider

provider: SQLAlchemyStructuredProvider = from_bind("salesforce")
tables = provider.models["dbo"]
session = provider.connect()

# Query provider session for data (location geoframes)

In [ ]:
import geojson
location_geoframes = [
    (
        row.Name,
        geojson.loads(
            row.JSON_String__c[:-1]
            if row.JSON_String__c[-1] == ","
            else row.JSON_String__c
        ),
    )
    for row in session.query(
        tables["GeoJSON_Location__c"].Name,
        tables["GeoJSON_Location__c"].JSON_String__c,
    )
    .join(
        tables["GeoJSON_Join__c"],
        tables["GeoJSON_Join__c"].GeoJSON_Location__c
        == tables["GeoJSON_Location__c"].Id,
    )
    .join(
        tables["Audience__c"],
        tables["Audience__c"].Id == tables["GeoJSON_Join__c"].Audience__c,
    )
    .join(
        tables["Child_ID__c"],
        tables["Child_ID__c"].Id == tables["Audience__c"].Child_ID__c,
    )
    .join(
        tables["Account"],
        tables["Account"].Id == tables["Child_ID__c"].Parent_Account__c,
    )
    .filter(
        tables["Account"].IsDeleted == False,
        tables["Child_ID__c"].IsDeleted == False,
        tables["Audience__c"].IsDeleted == False,
        tables["GeoJSON_Join__c"].IsDeleted == False,
        tables["GeoJSON_Location__c"].IsDeleted == False,
        tables["Account"].Account_Status__c == "Active",
        tables["Audience__c"].Active_Audience__c == True,
        tables["Audience__c"].Audience_Type__c == "Competitor Location",
        tables["GeoJSON_Join__c"].Active_Location__c == True,
    )
    .distinct()
]
len(location_geoframes)

In [ ]:
[f for f in location_geoframes if f[0] == "EF~33767"]

In [ ]:
from azure.storage.blob import ContainerClient

container: ContainerClient = ContainerClient.from_connection_string(
    conn_str=os.environ["ONSPOT_CONN_STR"], container_name="dashboard"
)

location_geoframes[2] = location_geoframes[0].str.replace("~", "")
location_geoframes[
    ~location_geoframes[2].isin(
        [
            p.split("/")[-1]
            for p in container.list_blob_names(
                name_starts_with="raw/2428875ee1674f548d47f05fa633119d/observations/"
            )
        ]
    )
][1].tolist()

# Format the OnSpot requests

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import uuid, os

# Relative date range
# now = datetime.utcnow()
# today = datetime(now.year, now.month, now.day)
# end = today - relativedelta(days=2)
# start = end - relativedelta(days=75)

# Static data range
start = datetime(2023,4,10)
end = datetime(2023,4,19)

requests = [
    {
        "type": "FeatureCollection",
        "features": [
            {
                **value,
                "properties": {
                    "name": key,
                    "fileName": key,
                    "start": start.isoformat(),
                    "end": end.isoformat(),
                    "hash": False,
                },
            }
            for key, value in location_geoframes
        ],
    },
]


ingress = {
    "instance_id": (instance_id := uuid.uuid4().hex),
    "conn_str": "ONSPOT_CONN_STR"
    if "ONSPOT_CONN_STR" in os.environ.keys()
    else "AzureWebJobsStorage",
    "container": os.environ.get("ONSPOT_CONTAINER", "dashboard"),
    "outputPath": f"oneoff/{instance_id}/devices",
    "endpoint": "/save/geoframe/all/devices",
}

# Inject callback and outputLocation data

In [ ]:
from azure.storage.blob import (
    ContainerClient,
    ContainerSasPermissions,
    generate_container_sas,
)

if ingress["endpoint"].startswith("/save/"):
    container = ContainerClient.from_connection_string(
        os.environ[ingress["conn_str"]]
        if ingress.get("conn_str", None) in os.environ.keys()
        else os.environ["AzureWebJobsStorage"],
        container_name=ingress.get("container", "general"),
    )
    if not container.exists():
        container.create_container()
    sas_token = generate_container_sas(
        account_name=container.credential.account_name,
        account_key=container.credential.account_key,
        container_name=container.container_name,
        permission=ContainerSasPermissions(write=True, read=True),
        expiry=datetime.utcnow() + relativedelta(days=2),
    )
event_url = "https://webhook.site/e8d0f8b4-25a9-483f-8273-4dddf5508c67"

for request in requests:
    if request.get("type", None) == "FeatureCollection":
        for feature in request["features"]:
            feature["properties"]["callback"] = event_url.replace(
                "{eventName}", uuid.uuid4().hex
            )
            if ingress["endpoint"].startswith("/save/"):
                feature["properties"]["outputLocation"] = (
                    container.url.replace("https://", "az://")
                    + "/{}?".format(ingress.get("outputPath", ingress["instance_id"]))
                    + sas_token
                )

# Iterate requests

In [ ]:
from libs.openapi.clients.onspot import OnSpot
req = OnSpot[(ingress["endpoint"], "post")]
for request in requests:
    display(req(request))